# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd2f01622e0>
├── 'a' --> tensor([[-0.7084,  1.0229, -0.1265],
│                   [-1.0901,  0.1787,  0.6028]])
└── 'x' --> <FastTreeValue 0x7fd2f015c970>
    └── 'c' --> tensor([[ 0.6395, -0.1356, -0.3477,  0.5625],
                        [ 0.4505,  0.1472,  0.9763,  0.0269],
                        [-0.8810,  0.9881, -0.7776,  0.7319]])

In [4]:
t.a

tensor([[-0.7084,  1.0229, -0.1265],
        [-1.0901,  0.1787,  0.6028]])

In [5]:
%timeit t.a

83.8 ns ± 3.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd2f01622e0>
├── 'a' --> tensor([[ 7.7900e-01, -1.9901e-03, -6.8406e-01],
│                   [-3.8094e-01, -5.2710e-01, -2.1424e+00]])
└── 'x' --> <FastTreeValue 0x7fd2f015c970>
    └── 'c' --> tensor([[ 0.6395, -0.1356, -0.3477,  0.5625],
                        [ 0.4505,  0.1472,  0.9763,  0.0269],
                        [-0.8810,  0.9881, -0.7776,  0.7319]])

In [7]:
%timeit t.a = new_value

116 ns ± 3.59 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7084,  1.0229, -0.1265],
               [-1.0901,  0.1787,  0.6028]]),
    x: Batch(
           c: tensor([[ 0.6395, -0.1356, -0.3477,  0.5625],
                      [ 0.4505,  0.1472,  0.9763,  0.0269],
                      [-0.8810,  0.9881, -0.7776,  0.7319]]),
       ),
)

In [10]:
b.a

tensor([[-0.7084,  1.0229, -0.1265],
        [-1.0901,  0.1787,  0.6028]])

In [11]:
%timeit b.a

80.9 ns ± 1.27 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7587,  1.9391,  0.6452],
               [ 0.6153,  0.7999, -0.7184]]),
    x: Batch(
           c: tensor([[ 0.6395, -0.1356, -0.3477,  0.5625],
                      [ 0.4505,  0.1472,  0.9763,  0.0269],
                      [-0.8810,  0.9881, -0.7776,  0.7319]]),
       ),
)

In [13]:
%timeit b.a = new_value

668 ns ± 14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.1 µs ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.7 µs ± 196 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

326 µs ± 20.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

327 µs ± 23.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd21a676100>
├── 'a' --> tensor([[[-0.7084,  1.0229, -0.1265],
│                    [-1.0901,  0.1787,  0.6028]],
│           
│                   [[-0.7084,  1.0229, -0.1265],
│                    [-1.0901,  0.1787,  0.6028]],
│           
│                   [[-0.7084,  1.0229, -0.1265],
│                    [-1.0901,  0.1787,  0.6028]],
│           
│                   [[-0.7084,  1.0229, -0.1265],
│                    [-1.0901,  0.1787,  0.6028]],
│           
│                   [[-0.7084,  1.0229, -0.1265],
│                    [-1.0901,  0.1787,  0.6028]],
│           
│                   [[-0.7084,  1.0229, -0.1265],
│                    [-1.0901,  0.1787,  0.6028]],
│           
│                   [[-0.7084,  1.0229, -0.1265],
│                    [-1.0901,  0.1787,  0.6028]],
│           
│                   [[-0.7084,  1.0229, -0.1265],
│                    [-1.0901,  0.1787,  0.6028]]])
└── 'x' --> <FastTreeValue 0x7fd2111bff40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

50.7 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd2f016ee50>
├── 'a' --> tensor([[-0.7084,  1.0229, -0.1265],
│                   [-1.0901,  0.1787,  0.6028],
│                   [-0.7084,  1.0229, -0.1265],
│                   [-1.0901,  0.1787,  0.6028],
│                   [-0.7084,  1.0229, -0.1265],
│                   [-1.0901,  0.1787,  0.6028],
│                   [-0.7084,  1.0229, -0.1265],
│                   [-1.0901,  0.1787,  0.6028],
│                   [-0.7084,  1.0229, -0.1265],
│                   [-1.0901,  0.1787,  0.6028],
│                   [-0.7084,  1.0229, -0.1265],
│                   [-1.0901,  0.1787,  0.6028],
│                   [-0.7084,  1.0229, -0.1265],
│                   [-1.0901,  0.1787,  0.6028],
│                   [-0.7084,  1.0229, -0.1265],
│                   [-1.0901,  0.1787,  0.6028]])
└── 'x' --> <FastTreeValue 0x7fd21a666ca0>
    └── 'c' --> tensor([[ 0.6395, -0.1356, -0.3477,  0.5625],
                        [ 0.4505,  0.1472,  0.9763,  0.0269],
                 

In [23]:
%timeit t_cat(trees)

47.9 µs ± 954 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

102 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.7084,  1.0229, -0.1265],
                [-1.0901,  0.1787,  0.6028]],
       
               [[-0.7084,  1.0229, -0.1265],
                [-1.0901,  0.1787,  0.6028]],
       
               [[-0.7084,  1.0229, -0.1265],
                [-1.0901,  0.1787,  0.6028]],
       
               [[-0.7084,  1.0229, -0.1265],
                [-1.0901,  0.1787,  0.6028]],
       
               [[-0.7084,  1.0229, -0.1265],
                [-1.0901,  0.1787,  0.6028]],
       
               [[-0.7084,  1.0229, -0.1265],
                [-1.0901,  0.1787,  0.6028]],
       
               [[-0.7084,  1.0229, -0.1265],
                [-1.0901,  0.1787,  0.6028]],
       
               [[-0.7084,  1.0229, -0.1265],
                [-1.0901,  0.1787,  0.6028]]]),
    x: Batch(
           c: tensor([[[ 0.6395, -0.1356, -0.3477,  0.5625],
                       [ 0.4505,  0.1472,  0.9763,  0.0269],
                       [-0.8810,  0.9881, -0.7776,  0.7319]],
         

In [26]:
%timeit Batch.stack(batches)

115 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.7084,  1.0229, -0.1265],
               [-1.0901,  0.1787,  0.6028],
               [-0.7084,  1.0229, -0.1265],
               [-1.0901,  0.1787,  0.6028],
               [-0.7084,  1.0229, -0.1265],
               [-1.0901,  0.1787,  0.6028],
               [-0.7084,  1.0229, -0.1265],
               [-1.0901,  0.1787,  0.6028],
               [-0.7084,  1.0229, -0.1265],
               [-1.0901,  0.1787,  0.6028],
               [-0.7084,  1.0229, -0.1265],
               [-1.0901,  0.1787,  0.6028],
               [-0.7084,  1.0229, -0.1265],
               [-1.0901,  0.1787,  0.6028],
               [-0.7084,  1.0229, -0.1265],
               [-1.0901,  0.1787,  0.6028]]),
    x: Batch(
           c: tensor([[ 0.6395, -0.1356, -0.3477,  0.5625],
                      [ 0.4505,  0.1472,  0.9763,  0.0269],
                      [-0.8810,  0.9881, -0.7776,  0.7319],
                      [ 0.6395, -0.1356, -0.3477,  0.5625],
                      [ 0.4505,  

In [28]:
%timeit Batch.cat(batches)

224 µs ± 8.25 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

746 µs ± 44.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
